# Final Project: Vehicle Rounting Problem

In [ ]:
import numpy as np
#np.set_printoptions(threshold=np.nan)
import seaborn as sns
from matplotlib import pyplot as plt
import math
import copy
import random
import timeit

## 1. Import, Data Investigation and Preprocessing

In [2]:
# VRP1
# import distance matrix
filepath = 'VRP1/distance.txt'
with open(filepath, 'r') as t:
    distance_matrix = []
    for line in t.readlines():
        y = [int(value) for value in line.split()]
        distance_matrix.append(y)
        
    t.close()
    distance_matrix = np.array(distance_matrix)

# import demand matrix
filepath = 'VRP1/demand.txt'
with open(filepath, 'r') as t:
    demand_matrix = []
    for line in t.readlines():
        y = [int(value) for value in line.split()]
        demand_matrix.append(y)
        
    t.close()
    demand_matrix = np.array(demand_matrix)
    demand_matrix = demand_matrix[0]
    # since demand matrix has length 100 but the distance_matrix has length 101, we add one demand with 0 to represent the depot 
    demand_matrix = np.insert(demand_matrix,0,0)

# import capacity matrix
filepath = 'VRP1/capacity.txt'
with open(filepath, 'r') as t:
    capacity_matrix = []
    for line in t.readlines():
        y = [int(value) for value in line.split()]
        capacity_matrix.append(y)
        
    t.close()
    capacity_matrix = np.array(capacity_matrix)
    capacity_matrix = capacity_matrix[0]

# import transportation_cost matrix
filepath = 'VRP1/transportation_cost.txt'
with open(filepath, 'r') as t:
    transportation_cost = []
    for line in t.readlines():
        y = [int(value) for value in line.split()]
        transportation_cost.append(y)
        
    t.close()
    transportation_cost = np.array(transportation_cost)
    transportation_cost = transportation_cost[0]
# VRP2
# import distance matrix
filepath = 'VRP2/distance.txt'
with open(filepath, 'r') as t:
    distance_matrix_2 = []
    for line in t.readlines():
        y = [int(value) for value in line.split()]
        distance_matrix_2.append(y)
        
    t.close()
    distance_matrix_2 = np.array(distance_matrix_2)

# import demand matrix
filepath = 'VRP2/demand.txt'
with open(filepath, 'r') as t:
    demand_matrix_2 = []
    for line in t.readlines():
        y = [int(value) for value in line.split()]
        demand_matrix_2.append(y)
        
    t.close()
    demand_matrix_2 = np.array(demand_matrix)
    demand_matrix_2 = demand_matrix_2[0]
    # since demand matrix has length 100 but the distance_matrix has length 101, we add one demand with 0 to represent the depot 
    demand_matrix_2 = np.insert(demand_matrix,0,0)

    
# import capacity matrix
filepath = 'VRP2/capacity.txt'
with open(filepath, 'r') as t:
    capacity_matrix_2 = []
    for line in t.readlines():
        y = [int(value) for value in line.split()]
        capacity_matrix_2.append(y)
        
    t.close()
    capacity_matrix_2 = np.array(capacity_matrix_2)
    capacity_matrix_2 = capacity_matrix_2[0]
    

    
# import transportation_cost matrix
filepath = 'VRP2/transportation_cost.txt'
with open(filepath, 'r') as t:
    transportation_cost_2 = []
    for line in t.readlines():
        y = [int(value) for value in line.split()]
        transportation_cost_2.append(y)
        
    t.close()
    transportation_cost_2 = np.array(transportation_cost_2)
    transportation_cost_2 = transportation_cost_2[0]

In [3]:
# information about the distance matrix for VRP1
print("Information about distance matric for VRP1 \n# columns: ", len(distance_matrix[0]))
print("# lines: ", len(distance_matrix))
asymmetric = (distance_matrix != np.transpose(distance_matrix))
print(np.unique(asymmetric))
print("Since we also get False values while comparing the matrix with its transposed, the matrix is asymmetric. \n")

# information about the distance matrix for VRP2
print("Information about distance matric for VRP2 \n# columns: ", len(distance_matrix_2[0]))
print("# lines: ", len(distance_matrix_2))
asymmetric_2 = (distance_matrix_2 != np.transpose(distance_matrix_2))
                
print(np.unique(asymmetric_2))
print("Since we also get False values while comparing the matrix with its transposed, the matrix is asymmetric.")

In [4]:
#heatmap
ax = sns.heatmap(distance_matrix, center=0)

The heatmap again illustrates the fact that we have to deal with an asymmetric distance matrix. In order to apply the k-means algorithm, we need a symmetric matrix. Therefore we need to make our distances symmetric be taking the mean between the distances (if they are different). 

## 2. Heuristic

In [5]:
# find the customer which is closest to another given customer
def find_closest_customer(last_customer_visited, customer_indices,distance_matrix):
    distance_list = []
    for i in customer_indices:
        if i == last_customer_visited:
            # set the distance to customer's own position very high in order for it not the be chosen as the closest customer
            distance_list.append(max(distance_matrix[last_customer_visited]))
        else: 
            distance_list.append(distance_matrix[i][last_customer_visited])
    

    index = distance_list.index(min(distance_list))
    customer_index = customer_indices[index]
    return customer_index      

In [6]:
def total_demand_satisfied(route,demand_matrix):
    total_demand_satisfied = 0
    for i in route:
        total_demand_satisfied += demand_matrix[i]
    return total_demand_satisfied

In [8]:
def get_cost_list(cars_used):
    
    '''
    gives us the list of the costs of the cars of the final clusters
    '''
    
    cars_capacity = []
    cars_capacity = list(cars_used.values())
    
    unique_capacity = total_cars()[0]
    unique_cost = total_cars()[1]
    cost_list = []
    
    for i in range(len(cars_capacity)):
        for j in range(len(unique_capacity)):
            if cars_capacity[i] == unique_capacity[j]:
                cost_list.append(unique_cost[j])
    
    return cost_list

In [9]:
# randomly pick a car from capacity matrix and return its capacity

def pick_random_car(capacity_matrix):
    car_type = random.randrange(len(capacity_matrix))
    cluster_size = capacity_matrix[car_type]
    return cluster_size

In [10]:
def cluster_initialization_with_random_cars(demand_matrix,distance_matrix,capacity_matrix):
    clusters = []
    cars_used = []

    customers = copy.deepcopy(demand_matrix)
    capacity = list(copy.deepcopy(capacity_matrix))
    customer_indices = list(range(len(demand_matrix))[1:])

    # keep on creating clusters as long as customers are not satisfied    
    while any(item != 0 for item in customers):
        
        # determine cluster size by randomly pick a car
        cluster_size = pick_random_car(capacity)
        
        # append capacity_size to the list of cars being used
        cars_used.append(cluster_size)
        
        # remove capacity that has been used from the capacity_matrix
        capacity.remove(cluster_size)
        
        if cluster_size not in [100,300,500,1000]:
            car_capacity = capacity_matrix[random.randrange(len(capacity_matrix))]
        else:
            car_capacity = cluster_size

        visited_customers = []

        # keep adding customers as long as capacities are not full
        while car_capacity > 0 and any(item != 0 for item in customers):
            
            # for the starting customer of each car, choose the customer that is farthest away from the depot
            if visited_customers == []:
                max_distance = -1
                max_distance_index = -1
                
                for index in customer_indices:
                    new_distance = distance_matrix[0][index]
                    if new_distance > max_distance:
                        max_distance = new_distance
                        max_distance_index = index
                    
                next_customer = max_distance_index
            # else append everytime the nearest customer
            else:
                last_customer = visited_customers[-1]
                next_customer = find_closest_customer(last_customer,customer_indices,distance_matrix)

            # after choosing next customer only merge if demand is completely covered
            if customers[next_customer] <= car_capacity:
                visited_customers.append(next_customer)
                
                # reduce the capacity by the customer demand
                car_capacity -= customers[next_customer]

                # demand of the customer is fully satsified
                customers[next_customer] = 0
                
                # delete visited customers
                del customer_indices[customer_indices.index(next_customer)]
            else:
                car_capacity = 0

        # create and add this car object to our current solution        
        clusters.append([0]+visited_customers+[0])

    return clusters, cars_used

In [11]:
def get_cost_list(cars_used):
    unique_capacity = sorted(set(capacity_matrix)) #VRP1: 100, 300, 500, 1000
    unique_cost = sorted(set(transportation_cost)) #VRP1: 11, 13, 15, 18

    cars_costs = dict(zip(unique_capacity, unique_cost))
    cost_list = [cars_costs.get(value) for value in cars_used]
    return cost_list

## 3. Genetic Algorithm

In [12]:
# get route and shuffle

def shuffle_cluster(cluster): 
    sliced_cluster = []
    sliced_cluster = cluster[1:len(cluster)-1]
    random.shuffle(sliced_cluster)
    sliced_cluster.append(0)
    sliced_cluster.insert(0,0)    
    return list(sliced_cluster)  

In [13]:
# create as many permutations as given with population_size. It returns a list in which every route is randomly
# shuffled n-times with n being the population size. 

def create_permutations(initialized_clusters, population_size):
    population_list = []
    sliced_shuffled = []
    
    for i in range(len(initialized_clusters)):
        for j in range(population_size):
            sliced_shuffled = shuffle_cluster(initialized_clusters[i])
            if sliced_shuffled not in population_list:
                population_list.append(sliced_shuffled)
            else: 
                unique = False
                while unique == False:
                    sliced_shuffled = shuffle_cluster(initialized_clusters[i])
                    if sliced_shuffled not in population_list:
                        population_list.append(sliced_shuffled)
                        unique = True
                    else: 
                        unique == False

    return population_list

In [14]:
def franzi_create_permutations(initialized_clusters, population_size):
    population_list = []
    sliced_shuffled = []
    
    
    for i in range(len(initialized_clusters)):
        population_list.append(initialized_clusters[i])
        for j in range(population_size-1):
            sliced_shuffled = shuffle_cluster(initialized_clusters[i])
            if sliced_shuffled not in population_list:
                population_list.append(sliced_shuffled)
            else: 
                unique = False
                while unique == False:
                    sliced_shuffled = shuffle_cluster(initialized_clusters[i])
                    if sliced_shuffled not in population_list:
                        population_list.append(sliced_shuffled)
                        unique = True
                    else: 
                        unique == False
    return population_list

### 3.1 Evaluate Fitness

Here the fitness of one cluster corresponds to the costs. Since we want to minimize our costs, a low evaluation value represents a high fitness. 

In [15]:
def evaluate_cluster(transportation_cost,distance_matrix,cluster):
    total_distance = 0
    
    for i in range(len(cluster)-1):
        total_cluster_distance = total_distance + distance_matrix[cluster[i]][cluster[i+1]]
        total_distance = total_cluster_distance
    return transportation_cost * total_distance

In [16]:
def total_distance_cluster(cluster):
    total_distance = 0
    
    for i in range(len(cluster)-1):
        total_cluster_distance = total_distance + distance_matrix[cluster[i]][cluster[i+1]]
        total_distance = total_cluster_distance
    return total_distance

In [17]:
def final_evaluation(list_of_clusters,cost_list):
    total_cost = 0 
    for i in range(len(list_of_clusters)):
        current_cluster = list_of_clusters[i]
        total_cost = total_cost + cost_list[i]*total_distance_cluster(current_cluster)
    return total_cost

### 3.2 Selection

The selection method we use is called tournament selection. It also takes the population (here: population_list), its fitness array (here:clusters costs) and the mating pool size (how many chromosomes do we want to keep) as input. It basically takes the chromosomes that have the lowest cost values. Note that highest fitness level corresponds to lowest cost values in this example since our aim is to minimize the overall costs.

In [18]:
def tournament_selection(population_list, mating_pool_size,population_size,cost_list):
    mating_pool_big = []
    counter = 0
    for i in range(len(population_list)):
        if i % population_size == 0: 
            current_population = population_list[i:i+population_size]
            mating_pool = []
            cost = cost_list[counter]
            counter +=1 
            j = 0
            #while(j < mating_pool_size):
            for j in range(mating_pool_size):
                participant1 = current_population[random.randint(0, population_size-1)]
                participant2 = current_population[random.randint(0, population_size-1)]
                
                if (participant1 != participant2):
                    j +=1
                    if evaluate_cluster(cost, distance_matrix, participant1) <= evaluate_cluster(cost, distance_matrix, participant2) :
                        mating_pool = mating_pool + [participant1]
                    else: 
                        mating_pool = mating_pool + [participant2]  
                else: 
                    mating_pool = mating_pool + [participant1] 
                    

            mating_pool_big = mating_pool_big + mating_pool 
    return mating_pool_big

### 3.3 Recombination

 

In recombination, two chromosomes are taken, called parent1 and parent2, and they are recombined (have children, here: child1 and child2) that might produce better outputs, i.e. lower cost.

In uniform_order_based_crossover, we use a template to create valid childs that still visit every customer on the route

In [19]:
def uniform_order_based_crossover(parent1,parent2):
    parent1 = parent1[1:-1]
    parent2 = parent2[1:-1]
    
    table = np.zeros(len(parent1))
    for i in range(len(parent1)):
        table[i]= random.randint(0,1)
    
    child1 = np.zeros(len(parent1))
    child2 = np.zeros(len(parent1))
    for i in range(len(table)):
        if table[i]==1:
            child1[i]=parent1[i]
            child2[i]=parent2[i]
        
    for i in range(len(table)):
        if child1[i]==0:
            for j in range(len(parent2)):
                if parent2[j] not in child1:
                    child1[i] = parent2[j]
                    break
        
        if child2[i]==0:
            for j in range(len(parent1)):
                if parent1[j] not in child2:
                    child2[i] = parent1[j]
                    break
                    
                
    child1 = np.insert(child1,0,0)
    child1 = np.insert(child1, len(child1),0)
    child2 = np.insert(child2,0,0)
    child2 = np.insert(child2, len(child2),0)
                    
    return child1, child2  

In [20]:
def recombine_uniform_order_based_crossover(population_size,mating_pool,crossover_probability, mating_pool_size):
    
    children_clusters_big = []
    
    for i in range(len(mating_pool)):
        if i % mating_pool_size == 0: 
            current_mating_pool = mating_pool[i:i+mating_pool_size]
            children_clusters = [] 

            j = 0
            while j < population_size:
                
                cluster1 = current_mating_pool[random.randint(0,mating_pool_size-1)]
                cluster2 = current_mating_pool[random.randint(0,mating_pool_size-1)]
                
                if cluster2 == cluster1:
                    cluster2 = current_mating_pool[random.randint(0,mating_pool_size-1)]
                    
                if random.random() < crossover_probability:
                    child1, child2 = uniform_order_based_crossover(cluster1,cluster2)
                    child1 = child1.astype(int)
                    child2 = child2.astype(int)
                    child1 = list(child1)
                    child2 = list(child2)
                    children_clusters.append(child1)
                    children_clusters.append(child2)
                    
                    
                    
                else:
                    #cluster1 = np.asarray(cluster1)
                    #cluster2 = np.asarray(cluster2)
                    children_clusters.append(cluster1)
                    children_clusters.append(cluster2)
                
                j+=2
                
            children_clusters_big = children_clusters_big + children_clusters

    return children_clusters_big

### 3.4 Swap Mutation

Mutation (in contrast to recombination) operates locally and tries to improve the fitness level with local optimization techniques. Here we implement the swap mutation, i.e. that given a particular probability, we swap at two randomly chosen locations within the cluster. Mutation makes sense in this example since we deal with an asymmetric matrix. This means that sometimes it is shorter to go from A to B instead from B to A. It is basically a form of permutation, which we have already done in the beginning for a fixed population_size. Nevertheless, we will still consider swap mutation since we will not be able to permutate over all customer nodes of one cluster, i.e. we still might be lucky to consider a better solution with the help of swap mutation.

In [21]:
def swap(cluster,mutation_probability):
    mutated_cluster = np.copy(cluster)
    if random.random() < mutation_probability:
        # note that we consider all entries without the first one and the last one since this represents the depot
        location1 = np.random.randint(1,len(cluster)-1)
        location2 = np.random.randint(1,len(cluster)-1)
        
        while location1 == location2:
            location2 = np.random.randint(1,len(cluster)-1)
        help_variable = mutated_cluster[location1]
        mutated_cluster[location1] = mutated_cluster[location2]
        mutated_cluster[location2] = help_variable
        
    return mutated_cluster

In [22]:
def mutate_swap(cluster,mutation_probability):
    all_mutated_clusters = []
    for i in range(len(cluster)):
        mutated_cluster = swap(cluster[i],mutation_probability)
        mutated_cluster = list(mutated_cluster)
        all_mutated_clusters.append(mutated_cluster)
    return all_mutated_clusters

In [23]:
# After last episode, choose best sequence for each cluster. 

def choose_best(all_populations, cost_list,population_size):
    final_clusters = []
    counter = 0
    
    for i in range(len(all_populations)):
        if i % population_size == 0: 
            current_population = all_populations[i:i+population_size]
            cost = cost_list[counter]
            counter += 1
            
            current_best = current_population[0]
            for j in range(population_size):
                if evaluate_cluster(cost, distance_matrix, current_population[j]) < evaluate_cluster(cost, distance_matrix, current_best):
                    current_best = current_population[j]
            final_clusters.append(current_best)
            #print("final", final_clusters)
            #print("best", current_best)
    
    return final_clusters

## 5. Run

In [24]:
population_size = 20
mating_pool_size = 8
crossover_probability = 0.7
mutation_probability = 0.05

starting_best, cars = cluster_initialization_with_random_cars(demand_matrix,distance_matrix,capacity_matrix)
best_cost_list = get_cost_list(cars)
for i in range(100):

    #start of genetic algorithm
    initialized_clusters, cars_used = cluster_initialization_with_random_cars(demand_matrix,distance_matrix,capacity_matrix)
    cost_list = get_cost_list(cars_used)

    print("Initialized Clusters:")
    print(initialized_clusters, '\n')
    print("Demand satisfied with clusters: ")
    for cluster in initialized_clusters:
        print(total_demand_satisfied(cluster,demand_matrix))
    print('cars used', cars_used)
    print('cost list', cost_list)
    print("cost_before_genetic", final_evaluation(initialized_clusters, cost_list))
    print("starting best", final_evaluation(starting_best, best_cost_list))

    if (final_evaluation(initialized_clusters, cost_list)) < (final_evaluation(starting_best, best_cost_list)): 
        starting_best = initialized_clusters
        best_cost_list = cost_list
        

all_populations = create_permutations(starting_best, population_size)
cost_list = best_cost_list
best = all_populations
for i in range(10000):

    mating_pool = tournament_selection(all_populations,mating_pool_size,population_size, cost_list)
    new_generation = recombine_uniform_order_based_crossover(population_size,mating_pool,crossover_probability, mating_pool_size)
    mutation = mutate_swap(new_generation, mutation_probability)
    all_populations = mutation
    print("i", i, final_evaluation(choose_best(all_populations, cost_list,population_size), cost_list))
    if (final_evaluation(choose_best(all_populations, cost_list,population_size), cost_list)) < final_evaluation(choose_best(best, cost_list,population_size), cost_list):
        best = all_populations
            

last_solution = choose_best(all_populations, cost_list,population_size)
final_solution = choose_best(best, cost_list,population_size)

print("last cost:", final_evaluation(last_solution, cost_list),"\n")
print("best cost:", final_evaluation(final_solution, cost_list))
print("car types used: ", cars_used)
print("Final route: ",choose_best(best, cost_list,population_size))

Initialized Clusters:
[[0, 68, 12, 73, 3, 4, 59, 29, 44, 51, 99, 75, 33, 85, 11, 77, 93, 5, 30, 92, 90, 65, 34, 80, 25, 0], [0, 96, 71, 48, 79, 45, 21, 83, 17, 98, 16, 74, 23, 63, 24, 40, 35, 0], [0, 18, 70, 82, 32, 0], [0, 6, 57, 47, 84, 72, 97, 42, 50, 49, 88, 60, 22, 86, 8, 61, 0], [0, 52, 20, 14, 37, 0], [0, 56, 78, 69, 46, 89, 0], [0, 36, 64, 27, 26, 67, 0], [0, 66, 2, 62, 31, 54, 0], [0, 28, 41, 76, 91, 7, 15, 10, 1, 39, 100, 9, 58, 55, 13, 94, 53, 81, 95, 43, 19, 38, 87, 0]] 

Demand satisfied with clusters: 
482
291
100
285
96
90
90
99
397
cars used [500, 300, 100, 300, 100, 100, 100, 100, 1000]
cost list [15, 13, 9, 13, 9, 9, 9, 9, 18]
cost_before_genetic 84719
starting best 93842
Initialized Clusters:
[[0, 68, 12, 73, 3, 0], [0, 33, 75, 85, 99, 51, 59, 4, 29, 44, 48, 79, 45, 21, 83, 0], [0, 96, 71, 18, 70, 82, 32, 56, 57, 47, 6, 97, 42, 50, 49, 88, 60, 72, 84, 89, 61, 86, 8, 22, 28, 41, 0], [0, 11, 77, 93, 5, 30, 92, 90, 65, 34, 80, 25, 14, 37, 87, 43, 35, 40, 31, 54, 81, 95,

126
cars used [300, 100, 100, 100, 500, 100, 100, 500, 100, 300]
cost list [13, 9, 9, 9, 15, 9, 9, 15, 9, 13]
cost_before_genetic 93200
starting best 81918
Initialized Clusters:
[[0, 68, 12, 73, 3, 0], [0, 33, 75, 85, 99, 51, 59, 4, 29, 44, 48, 79, 45, 21, 83, 17, 98, 16, 74, 23, 63, 24, 40, 35, 43, 95, 0], [0, 96, 71, 18, 70, 82, 32, 56, 57, 47, 6, 97, 42, 50, 49, 0], [0, 11, 77, 93, 5, 30, 0], [0, 92, 90, 65, 34, 80, 0], [0, 52, 20, 14, 25, 0], [0, 60, 88, 72, 84, 89, 61, 86, 8, 22, 28, 41, 67, 26, 27, 64, 36, 2, 0], [0, 78, 69, 46, 62, 31, 54, 81, 53, 94, 9, 58, 55, 13, 91, 76, 7, 15, 0], [0, 37, 87, 19, 38, 100, 0], [0, 66, 39, 1, 10, 0]] 

Demand satisfied with clusters: 
88
488
294
99
88
82
300
296
100
95
cars used [100, 500, 300, 100, 100, 100, 300, 300, 100, 100]
cost list [9, 15, 13, 9, 9, 9, 13, 13, 9, 9]
cost_before_genetic 87755
starting best 81918
Initialized Clusters:
[[0, 68, 12, 73, 3, 4, 59, 29, 44, 51, 99, 75, 33, 85, 0], [0, 96, 71, 48, 79, 45, 21, 83, 17, 98, 16, 74

Initialized Clusters:
[[0, 68, 12, 73, 3, 0], [0, 33, 75, 85, 99, 0], [0, 96, 71, 29, 44, 59, 4, 51, 48, 79, 45, 21, 83, 17, 98, 16, 74, 23, 63, 24, 40, 35, 43, 95, 81, 54, 31, 53, 94, 0], [0, 11, 77, 93, 5, 30, 0], [0, 18, 70, 82, 32, 0], [0, 6, 57, 47, 84, 72, 97, 42, 50, 49, 88, 60, 22, 86, 8, 61, 89, 46, 69, 78, 56, 36, 64, 27, 26, 67, 41, 28, 91, 76, 13, 55, 58, 9, 100, 39, 1, 10, 15, 7, 62, 2, 66, 19, 38, 87, 25, 14, 37, 34, 80, 90, 92, 0], [0, 65, 52, 20, 0]] 

Demand satisfied with clusters: 
88
91
497
99
100
982
73
cars used [100, 100, 500, 100, 100, 1000, 100]
cost list [9, 9, 15, 9, 9, 18, 9]
cost_before_genetic 98841
starting best 81918
Initialized Clusters:
[[0, 68, 12, 73, 3, 0], [0, 33, 75, 85, 99, 0], [0, 96, 71, 29, 44, 59, 4, 51, 48, 79, 45, 21, 83, 17, 98, 16, 74, 0], [0, 11, 77, 93, 5, 30, 92, 90, 65, 34, 80, 25, 14, 37, 87, 43, 35, 40, 31, 54, 81, 95, 53, 94, 9, 58, 55, 0], [0, 18, 70, 82, 32, 56, 57, 47, 6, 97, 42, 50, 49, 88, 60, 72, 84, 89, 61, 86, 8, 22, 28, 41

[[0, 68, 12, 73, 3, 4, 59, 29, 44, 51, 99, 75, 33, 85, 0], [0, 96, 71, 48, 79, 45, 0], [0, 11, 77, 93, 5, 30, 0], [0, 18, 70, 82, 32, 56, 57, 47, 6, 97, 42, 50, 49, 88, 60, 0], [0, 92, 90, 65, 34, 80, 0], [0, 52, 20, 14, 25, 0], [0, 21, 83, 17, 98, 16, 74, 23, 63, 24, 40, 35, 43, 95, 81, 54, 31, 53, 94, 9, 58, 55, 13, 91, 76, 7, 15, 10, 1, 39, 100, 38, 19, 87, 37, 62, 2, 66, 36, 64, 27, 26, 67, 41, 28, 22, 86, 8, 61, 89, 84, 72, 46, 69, 0], [0, 78, 0]] 

Demand satisfied with clusters: 
280
83
99
294
88
82
994
10
cars used [300, 100, 100, 300, 100, 100, 1000, 100]
cost list [13, 9, 9, 13, 9, 9, 18, 9]
cost_before_genetic 94967
starting best 76710
Initialized Clusters:
[[0, 68, 12, 73, 3, 0], [0, 33, 75, 85, 99, 51, 59, 4, 29, 44, 48, 79, 45, 21, 83, 17, 98, 16, 74, 23, 63, 24, 40, 35, 43, 95, 81, 54, 31, 53, 94, 9, 58, 55, 13, 91, 76, 7, 15, 10, 1, 39, 100, 38, 19, 87, 25, 14, 37, 34, 80, 5, 30, 0], [0, 96, 71, 18, 70, 82, 32, 56, 57, 47, 6, 97, 42, 50, 49, 0], [0, 11, 77, 93, 90, 92, 

Initialized Clusters:
[[0, 68, 12, 73, 3, 4, 59, 29, 44, 51, 99, 75, 33, 85, 0], [0, 96, 71, 48, 79, 45, 21, 83, 17, 98, 16, 74, 23, 63, 24, 40, 35, 0], [0, 11, 77, 93, 5, 30, 0], [0, 18, 70, 82, 32, 0], [0, 6, 57, 47, 84, 0], [0, 92, 90, 65, 34, 80, 25, 14, 37, 87, 43, 95, 81, 54, 31, 53, 94, 0], [0, 52, 20, 38, 19, 0], [0, 60, 49, 88, 50, 42, 97, 72, 89, 61, 86, 8, 22, 28, 41, 67, 26, 27, 64, 36, 2, 62, 66, 78, 69, 46, 56, 13, 0], [0, 91, 76, 7, 15, 10, 0], [0, 1, 39, 100, 9, 0], [0, 55, 58, 0]] 

Demand satisfied with clusters: 
280
291
99
100
77
279
75
496
91
95
47
cars used [300, 300, 100, 100, 100, 300, 100, 500, 100, 100, 100]
cost list [13, 13, 9, 9, 9, 13, 9, 15, 9, 9, 9]
cost_before_genetic 85976
starting best 76419
Initialized Clusters:
[[0, 68, 12, 73, 3, 4, 59, 29, 44, 51, 99, 75, 33, 85, 11, 77, 93, 5, 30, 92, 90, 65, 34, 80, 25, 0], [0, 96, 71, 48, 79, 45, 0], [0, 18, 70, 82, 32, 0], [0, 6, 57, 47, 84, 72, 97, 42, 50, 49, 88, 60, 22, 86, 8, 61, 0], [0, 52, 20, 14, 37, 87

...

i 7025 72888
i 7026 72888
i 7027 72888
i 7028 72888
i 7029 72888
i 7030 72888
i 7031 72888
i 7032 72888
i 7033 72888
i 7034 72888
i 7035 72888
i 7036 72888
i 7037 72888
i 7038 72888
i 7039 72888
i 7040 72888
i 7041 72888
i 7042 72888
i 7043 72888
i 7044 72888
i 7045 72888
i 7046 72888
i 7047 72888
i 7048 72888
i 7049 72888
i 7050 72888
i 7051 72888
i 7052 72888
i 7053 72888
i 7054 72888
i 7055 72888
i 7056 72888
i 7057 72888
i 7058 72888
i 7059 72888
i 7060 72888
i 7061 72888
i 7062 72888
i 7063 72888
i 7064 72888
i 7065 72888
i 7066 72888
i 7067 72888
i 7068 72888
i 7069 72888
i 7070 72888
i 7071 72888
i 7072 72888
i 7073 72888
i 7074 72888
i 7075 72888
i 7076 72888
i 7077 72888
i 7078 72888
i 7079 72888
i 7080 72888
i 7081 72888
i 7082 72888
i 7083 72888
i 7084 72888
i 7085 72888
i 7086 72888
i 7087 72888
i 7088 72888
i 7089 72888
i 7090 72888
i 7091 72888
i 7092 72888
i 7093 72888
i 7094 72888
i 7095 72888
i 7096 72888
i 7097 72888
i 7098 72888
i 7099 72888
i 7100 72888
i 7101 72888

i 7656 72828
i 7657 72828
i 7658 72828
i 7659 72828
i 7660 72828
i 7661 72828
i 7662 72828
i 7663 72828
i 7664 72828
i 7665 72828
i 7666 72828
i 7667 72828
i 7668 72828
i 7669 72828
i 7670 72828
i 7671 72828
i 7672 72828
i 7673 72828
i 7674 72828
i 7675 72828
i 7676 72828
i 7677 72828
i 7678 72828
i 7679 72828
i 7680 72828
i 7681 72828
i 7682 72828
i 7683 72828
i 7684 72828
i 7685 72828
i 7686 72828
i 7687 72828
i 7688 72828
i 7689 72828
i 7690 72828
i 7691 72828
i 7692 72828
i 7693 72828
i 7694 72828
i 7695 72828
i 7696 72828
i 7697 72828
i 7698 72828
i 7699 72828
i 7700 72828
i 7701 72828
i 7702 72828
i 7703 72828
i 7704 72828
i 7705 72828
i 7706 72828
i 7707 72828
i 7708 72828
i 7709 72828
i 7710 72828
i 7711 72828
i 7712 72828
i 7713 72828
i 7714 72828
i 7715 72828
i 7716 72828
i 7717 72828
i 7718 72828
i 7719 72828
i 7720 72828
i 7721 72828
i 7722 72828
i 7723 72828
i 7724 72828
i 7725 72828
i 7726 72828
i 7727 72828
i 7728 72828
i 7729 72828
i 7730 72828
i 7731 72828
i 7732 72828

i 8288 75468
i 8289 75468
i 8290 75468
i 8291 75468
i 8292 75468
i 8293 75468
i 8294 75468
i 8295 75468
i 8296 75468
i 8297 75468
i 8298 73998
i 8299 74358
i 8300 74358
i 8301 74508
i 8302 74358
i 8303 74508
i 8304 74508
i 8305 74508
i 8306 74508
i 8307 74508
i 8308 74508
i 8309 74508
i 8310 74508
i 8311 74508
i 8312 74508
i 8313 74508
i 8314 74508
i 8315 74508
i 8316 74508
i 8317 74508
i 8318 74508
i 8319 74508
i 8320 74508
i 8321 74508
i 8322 74508
i 8323 74508
i 8324 74508
i 8325 74358
i 8326 74508
i 8327 74508
i 8328 74508
i 8329 74508
i 8330 74508
i 8331 74508
i 8332 74508
i 8333 74508
i 8334 74508
i 8335 74508
i 8336 74508
i 8337 74508
i 8338 74508
i 8339 74508
i 8340 74508
i 8341 74508
i 8342 74508
i 8343 74508
i 8344 74508
i 8345 74508
i 8346 74508
i 8347 74508
i 8348 74508
i 8349 74508
i 8350 74508
i 8351 74508
i 8352 74508
i 8353 74508
i 8354 74508
i 8355 74508
i 8356 74508
i 8357 74508
i 8358 74508
i 8359 74508
i 8360 74508
i 8361 74508
i 8362 74508
i 8363 74508
i 8364 74508

i 8919 74004
i 8920 74004
i 8921 74004
i 8922 74004
i 8923 74004
i 8924 74004
i 8925 74004
i 8926 74004
i 8927 74004
i 8928 74004
i 8929 74004
i 8930 74004
i 8931 74004
i 8932 74004
i 8933 74004
i 8934 74004
i 8935 74004
i 8936 74004
i 8937 74004
i 8938 74004
i 8939 74004
i 8940 74004
i 8941 74004
i 8942 74004
i 8943 74004
i 8944 74004
i 8945 74004
i 8946 74004
i 8947 74004
i 8948 74004
i 8949 74004
i 8950 74004
i 8951 74004
i 8952 74004
i 8953 74004
i 8954 74004
i 8955 74004
i 8956 74004
i 8957 74004
i 8958 74004
i 8959 74004
i 8960 74004
i 8961 74004
i 8962 74004
i 8963 74004
i 8964 74004
i 8965 74004
i 8966 74004
i 8967 74004
i 8968 74004
i 8969 74004
i 8970 74004
i 8971 74004
i 8972 74004
i 8973 74004
i 8974 74004
i 8975 74004
i 8976 74004
i 8977 74004
i 8978 74004
i 8979 74004
i 8980 74004
i 8981 74004
i 8982 74004
i 8983 74004
i 8984 74004
i 8985 74004
i 8986 74004
i 8987 74004
i 8988 74004
i 8989 74004
i 8990 74004
i 8991 74004
i 8992 74004
i 8993 74004
i 8994 74004
i 8995 74004

i 9551 74004
i 9552 74004
i 9553 74004
i 9554 74004
i 9555 74004
i 9556 74004
i 9557 74004
i 9558 74004
i 9559 74004
i 9560 74004
i 9561 74004
i 9562 74004
i 9563 74004
i 9564 74004
i 9565 74004
i 9566 74004
i 9567 74004
i 9568 74004
i 9569 74004
i 9570 74004
i 9571 74004
i 9572 74004
i 9573 74004
i 9574 74004
i 9575 74004
i 9576 74004
i 9577 74004
i 9578 74004
i 9579 74004
i 9580 74004
i 9581 74004
i 9582 74004
i 9583 74004
i 9584 74004
i 9585 74004
i 9586 74004
i 9587 74004
i 9588 74004
i 9589 74004
i 9590 74004
i 9591 74004
i 9592 74004
i 9593 74004
i 9594 74004
i 9595 74004
i 9596 74004
i 9597 74004
i 9598 74004
i 9599 74004
i 9600 74004
i 9601 74004
i 9602 74004
i 9603 74004
i 9604 74004
i 9605 74004
i 9606 74004
i 9607 74004
i 9608 74004
i 9609 74004
i 9610 74004
i 9611 74004
i 9612 74004
i 9613 74004
i 9614 74004
i 9615 74004
i 9616 74004
i 9617 74004
i 9618 74004
i 9619 74004
i 9620 74004
i 9621 74004
i 9622 74004
i 9623 74004
i 9624 74004
i 9625 74004
i 9626 74004
i 9627 74004